In [ ]:
from Utils.my_utils import *
from Recommenders.GraphBased.RP3betaRecommender import *
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.NonPersonalizedRecommender import *
import os

In [ ]:
# seed to be set
seed = 480
np.random.seed(seed)
users, items, users_to_eval = readData()
URM, user_originalID_to_index, item_originalID_to_index, item_index_to_originalID = preProcess()
URM_train_full, URM_train, URM_test, URM_validation = splitURM(URM)

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
output_folder_path = "result_best_models/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [ ]:
# load the model with all best parameters
recommender_object = RP3betaRecommender(URM_train_full)
recommender_object.load_model(output_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip")

In [ ]:
# define and load recommender
recommender_object_top_pop = TopPop(URM_train_full)
recommender_object_top_pop.load_model(output_folder_path, file_name = recommender_object_top_pop.RECOMMENDER_NAME + "_my_own_save.zip")

In [ ]:
# recommend items to users
rec_items_ind = []
for user_id in users_to_eval:
    if(user_id in users):
        # recommend users for which we already have some ratings
        rec_items_ind.append(recommender_object.recommend(user_originalID_to_index[user_id], cutoff=10))
    else:
        # recommend users with no stored rating
        rec_items_ind.append(recommender_object_top_pop.recommend(user_id, cutoff=10, remove_seen_flag=False))

In [ ]:
# evaluate recommender
evaluate_algorithm(URM_test, recommender_object)

In [ ]:
storeRecommendations(users_to_eval, rec_items_ind, item_index_to_originalID)